In [4]:
from architectures_v1 import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, RNN, LSTM
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import to_categorical

import os
import numpy as np
import tensorflow as tf
import pandas as pd
import time

## Loading Training/Testing/Validation Data

In [2]:
dataset = 'PMNIST'

x_train_load = np.load(os.path.abspath(os.path.join('99_data',dataset,'x_train.npz')))
x_test_load = np.load(os.path.abspath(os.path.join('99_data',dataset,'x_test.npz')))
x_valid_load = np.load(os.path.abspath(os.path.join('99_data',dataset,'x_valid.npz')))

x_train = x_train_load['arr_0']
x_test = x_test_load['arr_0']
x_valid = x_valid_load['arr_0']

y_train_load = np.load(os.path.abspath(os.path.join('99_data',dataset,'y_train.npz')))
y_test_load = np.load(os.path.abspath(os.path.join('99_data',dataset,'y_test.npz')))
y_valid_load = np.load(os.path.abspath(os.path.join('99_data',dataset,'y_valid.npz')))

y_train = y_train_load['arr_0']
y_test = y_test_load['arr_0']
y_valid = y_valid_load['arr_0']

print(f"x_train: {x_train.shape} - y_train: {y_train.shape}")
print(f"x_test: {x_test.shape} - y_test: {y_test.shape}")
print(f"x_valid: {x_valid.shape} - y_valid: {y_valid.shape}")

x_train: (50000, 784, 1) - y_train: (50000,)
x_test: (10000, 784, 1) - y_test: (10000,)
x_valid: (10000, 784, 1) - y_valid: (10000,)


In [6]:
seed = 0
tf.random.set_random_seed(seed)
np.random.seed(seed)

length = x_train.shape[1]
n_features = x_train.shape[-1]

model = Sequential()

model.add(LSTM(units = 212, 
              input_shape = (length, n_features), return_sequences = False))
model.add(Dense(10, activation = "softmax"))

model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 212)               181472    
_________________________________________________________________
dense (Dense)                (None, 10)                2130      
Total params: 183,602
Trainable params: 183,602
Non-trainable params: 0
_________________________________________________________________


In [ ]:
if not os.path.exists(os.path.abspath(os.path.join('models', dataset))):
    os.mkdir(os.path.abspath(os.path.join('models', dataset)))

file_path = os.path.abspath(os.path.join('models', dataset, f'{dataset}-LMU.hdf5'))

callbacks = [
    ModelCheckpoint(filepath=file_path, monitor='val_loss', save_best_only = True, mode = 'min', verbose = 1), 
    EarlyStopping(monitor = 'val_loss', patience = 10, mode = 'min')]

t = time.time()

result = model.fit(x_train, 
                   to_categorical(y_train),
                   epochs = 200, 
                   batch_size = 100, 
                   validation_data = (x_valid, to_categorical(y_valid)), 
                   callbacks = callbacks)

print(f"Training time: {time.time() - t} s")

df_results = pd.DataFrame(result.history)
df_results.to_csv(os.path.abspath(os.path.join('models', dataset, 'LMU_results.csv')))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 50000 samples, validate on 10000 samples
Epoch 1/200
49900/50000 [============================>.] - ETA: 0s - loss: 2.0446 - acc: 0.2663
Epoch 00001: val_loss improved from inf to 1.65045, saving model to /data/home/dorads/Documents/GitHub_Repositories/01_Maintained/NeuralODE/models/PMNIST/PMNIST-LMU.hdf5
50000/50000 [==============================] - 386s 8ms/sample - loss: 2.0435 - acc: 0.2668 - val_loss: 1.6504 - val_acc: 0.4291
Epoch 2/200
49900/50000 [============================>.] - ETA: 0s - loss: 1.5692 - acc: 0.4535
Epoch 00002: val_loss improved from 1.65045 to 1.29461, saving model to /data/home/dorads/Documents/GitHub_Repositories/01_Maintained/NeuralODE/models/PMNIST/PMNIST-LMU.hdf5
50000/50000 [==============================] - 385s 8ms/sample - loss: 1.5683 - acc: 0.4538 - val_loss: 1.2946 - val_acc: 0.5597
Epoch 3/200
49900/50000 [============================>.] - ETA

In [3]:
model.load_weights(file_path)
model.evaluate(x_test, to_categorical(y_test))

NameError: name 'model' is not defined